<a href="https://colab.research.google.com/github/yy0750/Analyzing_Depression_Emotions/blob/main/Analyzing_Depression_Emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyzing Depression Emotions



# Module Import

In [ ]:
!pip install konlpy
!pip install squarify #트리맵

In [ ]:
# # 폰트 설치
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
import squarify
from wordcloud import WordCloud

In [ ]:
plt.rc('font', family='NanumBarunGothic') #폰트를 나눔 고딕 폰트로 지정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data Load

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/감정분석/data.csv")

df['감정_대분류'] = df['감정_대분류'].str.strip()
unique_emotions = df['감정_대분류'].unique()
df['사람문장'] = df['사람문장1'] + df['사람문장2']
print(df.shape)
df.head()

In [ ]:
# 불용어 리스트
#stopwords = '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','아','휴','아이구','아이쿠','아이고','어','나','우리','저희','따라','의해','을','를','에','의','가','으로','로','에게','뿐이다','의거하여','근거하여','입각하여','기준으로','예하면','예를 들면','예를 들자면','저','소인','소생','저희','지말고','하지마','하지마라','다른','물론','또한','그리고','비길수 없다','해서는 안된다','뿐만 아니라','만이 아니다','만은 아니다','막론하고','관계없이','그치지 않다','그러나','그런데','하지만','든간에','논하지 않다','따지지 않다','설사','비록','더라도','아니면','만 못하다','하는 편이 낫다','불문하고','향하여','향해서','향하다','쪽으로','틈타','이용하여','타다','오르다','제외하고','이 외에','이 밖에','하여야','비로소','한다면 몰라도','외에도','이곳','여기','부터','기점으로','따라서','할 생각이다','하려고하다','이리하여','그리하여','그렇게 함으로써','하지만','일때','할때','앞에서','중에서','보는데서','으로써','로써','까지','해야한다','일것이다','반드시','할줄알다','할수있다','할수있어','임에 틀림없다','한다면','등','등등','제','겨우','단지','다만','할뿐','딩동','댕그','대해서','대하여','대하면','훨씬','얼마나','얼마만큼','얼마큼','남짓','여','얼마간','약간','다소','좀','조금','다수','몇','얼마','지만','하물며','또한','그러나','그렇지만','하지만','이외에도','대해 말하자면','뿐이다','다음에','반대로','반대로 말하자면','이와 반대로','바꾸어서 말하면','바꾸어서 한다면','만약','그렇지않으면','까악','툭','딱','삐걱거리다','보드득','비걱거리다','꽈당','응당','해야한다','에 가서','각','각각','여러분','각종','각자','제각기','하도록하다','와','과','그러므로','그래서','고로','한 까닭에','하기 때문에','거니와','이지만','대하여','관하여','관한','과연','실로','아니나다를가','생각한대로','진짜로','한적이있다','하곤하였다','하','하하','허허','아하','거바','와','오','왜','어째서','무엇때문에','어찌','하겠는가','무슨','어디','어느곳','더군다나','하물며','더욱이는','어느때','언제','야','이봐','어이','여보시오','흐흐','흥','휴','헉헉','헐떡헐떡','영차','여차','어기여차','끙끙','아야','앗','아야','콸콸','졸졸','좍좍','뚝뚝','주룩주룩','솨','우르르','그래도','또','그리고','바꾸어말하면','바꾸어말하자면','혹은','혹시','답다','및','그에 따르는','때가 되어','즉','지든지','설령','가령','하더라도','할지라도','일지라도','지든지','몇','거의','하마터면','인젠','이젠','된바에야','된이상','만큼','어찌됏든','그위에','게다가','점에서 보아','비추어 보아','고려하면','하게될것이다','일것이다','비교적','좀','보다더','비하면','시키다','하게하다','할만하다','의해서','연이서','이어서','잇따라','뒤따라','뒤이어','결국','의지하여','기대여','통하여','자마자','더욱더','불구하고','얼마든지','마음대로','주저하지 않고','곧','즉시','바로','당장','하자마자','밖에 안된다','하면된다','그래','그렇지','요컨대','다시 말하자면','바꿔 말하면','즉','구체적으로','말하자면','시작하여','시초에','이상','허','헉','허걱','바와같이','해도좋다','해도된다','게다가','더구나','하물며','와르르','팍','퍽','펄렁','동안','이래','하고있었다','이었다','에서','로부터','까지','예하면','했어요','해요','함께','같이','더불어','마저','마저도','양자','모두','습니다','가까스로','하려고하다','즈음하여','다른','다른 방면으로','해봐요','습니까','했어요','말할것도 없고','무릎쓰고','개의치않고','하는것만 못하다','하는것이 낫다','매','매번','들','모','어느것','어느','로써','갖고말하자면','어디','어느쪽','어느것','어느해','어느 년도','라 해도','언젠가','어떤것','어느것','저기','저쪽','저것','그때','그럼','그러면','요만한걸','그래','그때','저것만큼','그저','이르기까지','할 줄 안다','할 힘이 있다','너','너희','당신','어찌','설마','차라리','할지언정','할지라도','할망정','할지언정','구토하다','게우다','토하다','메쓰겁다','옆사람','퉤','쳇','의거하여','근거하여','의해','따라','힘입어','그','다음','버금','두번째로','기타','첫번째로','나머지는','그중에서','견지에서','형식으로 쓰여','입장에서','위해서','단지','의해되다','하도록시키다','뿐만아니라','반대로','전후','전자','앞의것','잠시','잠깐','하면서','그렇지만','다음에','그러한즉','그런즉','남들','아무거나','어찌하든지','같다','비슷하다','예컨대','이럴정도로','어떻게','만약','만일','위에서 서술한바와같이','인 듯하다','하지 않는다면','만약에','무엇','무슨','어느','어떤','아래윗','조차','한데','그럼에도 불구하고','여전히','심지어','까지도','조차도','하지 않도록','않기 위하여','때','시각','무렵','시간','동안','어때','어떠한','하여금','네','예','우선','누구','누가 알겠는가','아무도','줄은모른다','줄은 몰랏다','하는 김에','겸사겸사','하는바','그런 까닭에','한 이유는','그러니','그러니까','때문에','그','너희','그들','너희들','타인','것','것들','너','위하여','공동으로','동시에','하기 위하여','어찌하여','무엇때문에','붕붕','윙윙','나','우리','엉엉','휘익','윙윙','오호','아하','어쨋든','만 못하다','하기보다는','차라리','하는 편이 낫다','흐흐','놀라다','상대적으로 말하자면','마치','아니라면','쉿','그렇지 않으면','그렇지 않다면','안 그러면','아니었다면','하든지','아니면','이라면','좋아','알았어','하는것도','그만이다','어쩔수 없다','하나','일','일반적으로','일단','한켠으로는','오자마자','이렇게되면','이와같다면','전부','한마디','한항목','근거로','하기에','아울러','하지 않도록','않기 위해서','이르기까지','이 되다','로 인하여','까닭으로','이유만으로','이로 인하여','그래서','이 때문에','그러므로','그런 까닭에','알 수 있다','결론을 낼 수 있다','으로 인하여','있다','어떤것','관계가 있다','관련이 있다','연관되다','어떤것들','에 대해','이리하여','그리하여','여부','하기보다는','하느니','하면 할수록','운운','이러이러하다','하구나','하도다','다시말하면','다음으로','에 있다','에 달려 있다','우리','우리들','오히려','하기는한데','어떻게','어떻해','어찌됏어','어때','어째서','본대로','자','이','이쪽','여기','이것','이번','이렇게말하자면','이런','이러한','이와 같은','요만큼','요만한 것','얼마 안 되는 것','이만큼','이 정도의','이렇게 많은 것','이와 같다','이때','이렇구나','것과 같이','끼익','삐걱','따위','와 같은 사람들','부류의 사람들','왜냐하면','중의하나','오직','오로지','에 한하다','하기만 하면','도착하다','까지 미치다','도달하다','정도에 이르다','할 지경이다','결과에 이르다','관해서는','여러분','하고 있다','한 후','혼자','자기','자기집','자신','우에 종합한것과같이','총적으로 보면','총적으로 말하면','총적으로','대로 하다','으로서','참','그만이다','할 따름이다','쿵','탕탕','쾅쾅','둥둥','봐','봐라','아이야','아니','와아','응','아이','참나','년','월','일','령','영','일','이','삼','사','오','육','륙','칠','팔','구','이천육','이천칠','이천팔','이천구','하나','둘','셋','넷','다섯','여섯','일곱','여덟','아홉','령','영'
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
             '아','휴','아이구','아이쿠','아이고','어','나','우리','저희','따라','의해','을','를','에','의','가',
             '으로','로','에게','뿐이다','의거하여','근거하여','입각하여','기준으로','예하면','예를 들면','예를 들자면']

# 한글과 공백을 제외한 모든 문자를 제거
def clean_text(text):
    text = re.sub(r'[^가-힣\s]', '', text)
    return text

# 형태소 추출 및 불용어 제거
def tokenize_text(text):
    okt = Okt()
    return ' '.join([word for word in okt.morphs(text) if word not in stopwords])



# 텍스트 전처리 및 특성 추출
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['사람문장'].apply(lambda x: tokenize_text(clean_text(x))))

# 연령 원-핫 인코딩
onehot_encoder = OneHotEncoder()
age_encoded = onehot_encoder.fit_transform(df[['연령']])

# 데이터 합치기
X = hstack([tfidf_matrix, age_encoded])
y = df['감정_대분류']

# 모델 학습 및 최적화
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(random_state=0, max_iter=1000)
params = {'C': [1, 5, 10]}  # 'C' 파라미터 조정
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(X_train, y_train)

##### 문장 입력 및 예측

In [ ]:
input_sentence = input('문장을 입력해 주세요: ')
input_age = input('연령대를 입력해 주세요 (예: 청소년, 청년, 중년, 노년): ')

input_sentence_cleaned = clean_text(input_sentence)
input_sentence_tokenized = tokenize_text(input_sentence_cleaned)
input_sentence_tfidf = tfidf_vectorizer.transform([input_sentence_tokenized])
input_age_encoded = onehot_encoder.transform([[input_age]])

combined_input = hstack([input_sentence_tfidf, input_age_encoded])

predicted_emotion = grid_cv.best_estimator_.predict(combined_input)
print(f'입력하신 내용에서 "{predicted_emotion[0]}" 감정이 느껴집니다.')

## Performance

In [ ]:
y_pred = grid_cv.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

## Precision, Recall, F1 Score

In [ ]:
print(classification_report(y_test, y_pred))

# 단어 빈도에 따른 시각화

##### 명사 추출

In [ ]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')
    result = hangul.sub('', text)
    okt = Okt()
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한 글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]
    return nouns


df['cleaned_nouns'] = df['사람문장'].apply(text_cleaning)

all_nouns = sum(df['cleaned_nouns'], [])

word_count = Counter(all_nouns)

##### 바 그래프

In [ ]:
top_n = 20
most_common_words = word_count.most_common(top_n)

words = [word[0] for word in most_common_words]
counts = [word[1] for word in most_common_words]
plt.figure(figsize=(10, 6))
plt.bar(words, counts)
plt.xlabel('단어')
plt.ylabel('빈도수')
plt.title(f'Top {top_n} Most Frequent Words')
plt.show()

##### 트리맵

In [ ]:
most_common_words = word_count.most_common(top_n)


words = [word[0] for word in most_common_words]
sizes = [word[1] for word in most_common_words]

# 트리맵 생성
plt.figure(figsize=(12, 8))
squarify.plot(sizes=sizes, label=words, alpha=0.8)
plt.axis('off')
plt.title(f'Top {top_n} Most Frequent Words TreeMap')
plt.show()

##### 워드클라우드

In [ ]:


top_n = 20
most_common_words = word_count.most_common(top_n)
word_freq = {word[0]: word[1] for word in most_common_words}

# 워드클라우드 생성
wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
               background_color='black',
               width=800, height=600)
wc.generate_from_frequencies(word_freq)

# 워드클라우드 시각화
plt.figure(figsize=(10, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')  # 축을 보이지 않게 설정
plt.title(f'Top {top_n} Most Frequent Words Word Cloud')
plt.show()


##### 연령대별 우울 감정 빈도

In [ ]:
age_groups = ['청소년', '청년', '중년', '노년']
age_group_names = ['Adolescents', 'Youth', 'Middle-aged', 'Older']
top_n = 20


age_group_word_freqs = {}

for age, name in zip(age_groups, age_group_names):
    filtered_df = df[(df['연령'] == age) & (df['감정_소분류'] == '우울한')]

    corpus = " ".join(filtered_df['사람문장'].tolist())

    cleaned_words = text_cleaning(corpus)
    word_counter = Counter(cleaned_words)

    age_group_word_freqs[name] = word_counter.most_common(top_n)

fig, axes = plt.subplots(len(age_groups), 1, figsize=(10, 20))

for ax, (name, common_words) in zip(axes, age_group_word_freqs.items()):
    words = [word[0] for word in common_words]
    counts = [word[1] for word in common_words]
    ax.bar(words, counts)
    ax.set_title(f'Top {top_n} words for {name}')
    ax.set_ylabel('Frequency')
    ax.set_xlabel('Words')

plt.tight_layout()
plt.show()
